In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sys
from itertools import groupby
import numpy as np
import cv2
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
from glob import glob
from multiprocessing import Pool
import matplotlib.pyplot as plt
# import cupy as cp
import ast
from pathlib import Path
import pickle5 as pickle
import torch

import shutil
import sys
sys.path.append("../src/")
sys.path.append("../yolov5/")

import util

from joblib import Parallel, delayed

from IPython.display import display, HTML
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from ensemble_boxes import weighted_boxes_fusion

import albumentations as A

from IPython.display import Image
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

def tracking_function(tracker, frame_id, bboxes, scores, best_conf, num_prev_bbox = None):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            # remember to check
            if scores[i] <= best_conf:
                continue
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    to_add_preds = []
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))

        #exclude those in the edge
        if (x_min + bbox_width >= 1279) or (y_min + bbox_height) >= 719 or (x_min <= 1) or (y_min <= 1):
            continue
        score = tobj.last_detection.scores[0]            
        to_add_preds.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        #predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))            
        
    if (num_prev_bbox is None or (num_prev_bbox - len(predictions) + 1) >= len(to_add_preds)):
        predictions.extend(to_add_preds)
        
    return predictions

In [ ]:
from ensemble_boxes import weighted_boxes_fusion
def run_wbf(bboxes, confs, iou_thr=0.5, skip_box_thr=0.00000001):
    if len(bboxes) == 1:
        return bboxes[0], confs[0], []   
    else:
        for i in range(len(bboxes)):
            sub_bboxes = bboxes[i]
            if len(sub_bboxes) > 0:
                sub_bboxes[:,2:] = sub_bboxes[:,2:] + sub_bboxes[:,:2]
                sub_bboxes[:,0] = sub_bboxes[:,0] / 1279.
                sub_bboxes[:,1] = sub_bboxes[:,1] / 719.
                sub_bboxes[:,2] = sub_bboxes[:,2] / 1279.
                sub_bboxes[:,3] = sub_bboxes[:,3] / 719.
            bboxes[i] = sub_bboxes    
    labels = [np.ones(len(conf)) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(bboxes, confs, labels, iou_thr=iou_thr, skip_box_thr=0.001, allows_overflow=True, conf_type='avg')
    if len(boxes) > 0:
        boxes[:,0] = boxes[:,0] * 1279.
        boxes[:,1] = boxes[:,1] * 719.
        boxes[:,2] = boxes[:,2] * 1279.
        boxes[:,3] = boxes[:,3] * 719.
        boxes[:,2:] = boxes[:,2:] - boxes[:,:2]
        
    boxes = [box for i,box in enumerate(boxes) if scores[i] >= skip_box_thr]
    scores = [conf for conf in scores if conf >= skip_box_thr]

    return boxes, scores, labels

In [ ]:
from IPython.display import Image
from PIL import Image as Img
import subprocess
#This code I found in: https://www.kaggle.com/bamps53/create-annotated-video Thank you for sharing.
out_dir = "/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/video_check/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

def load_image(image_dir):
    assert os.path.exists(image_dir), f'{image_dir} does not exist.'
    img = cv2.imread(str(image_dir))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def load_image_with_annotations(image_dir, annotations):
    img = load_image(image_dir)
    if len(annotations) > 0:
        for ann in annotations:
            cv2.rectangle(img, (ann['x'], ann['y']),
                (ann['x'] + ann['width'], ann['y'] + ann['height']),
                (0, 255, 0), thickness=3)
    return img

def show_prediction(img, bboxes, scores, show = True):
    colors = [(0, 0, 255)]

    obj_names = ["s"]

    for box, score in zip(bboxes, scores):
        cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[0] + box[2]), int(box[1] + box[3])), (255,0,0), 2)
        cv2.putText(img, f'{score:.2f}', (int(box[0]), int(box[1])-3), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2, cv2.LINE_AA)
    if show:
        img = Img.fromarray(img).resize((1280, 720))
    return img

def make_video(df, video_id, sequence_id, pred_col, out_dir, best_conf, name=None, s_f=None, e_f=None):
    fps = 15
    width = 1280
    height = 720
    if name is None:
        name = "video"
    
    save_path = f'{out_dir}/{name}.mp4'
    tmp_path =  f'{out_dir}/tmp-{name}.mp4'
    output_video = cv2.VideoWriter(tmp_path, cv2.VideoWriter_fourcc(*"MP4V"), fps, (width, height),)
    
    # I just generate ony part of video
    if video_id is not None and sequence_id is not None:
        query = 'video_id == {} and sequence == {}'.format(video_id, sequence_id)
        if s_f is not None:
            query = query + 'and video_frame >= {} and video_frame <= {}'.format(s_f, e_f)
        print(query)
        video_df = df.query(query)
    else:
        video_df = df
    print(video_df.shape)
    for _, row in tqdm(video_df.iterrows(), total=len(video_df)):
        video_id = row.video_id
        video_frame = row.video_frame
        annotations = row.annotations
        img_file = row.image_path
        img = load_image_with_annotations(img_file, annotations)
        preds = row[pred_col]
        best_conf = 0
        bboxes = [preds[i][1:] for i in range(len(preds)) if preds[i][0] >= best_conf]
        confis = [preds[i][0] for i in range(len(preds)) if preds[i][0] >= best_conf]        
        img = show_prediction(img, bboxes, confis, show=False)
        cv2.putText(img, f'{video_id}-{video_frame}', (10,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1, cv2.LINE_AA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        output_video.write(img)
    
    output_video.release()

    if os.path.exists(save_path):
        os.remove(save_path)
    subprocess.run(
        ["ffmpeg", "-i", tmp_path, "-crf", "18", "-preset", "veryfast", "-vcodec", "libx264", save_path]
    )
    os.remove(tmp_path)

In [ ]:
from base64 import b64encode
def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1280 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

# Read Data

In [ ]:
INPUT_DIR = Path("../../data/tensorflow-great-barrier-reef/")
df_origin = pd.read_csv(INPUT_DIR / "train.csv")

In [ ]:

df = pd.read_csv(INPUT_DIR / "train.csv")
folds = util.load_pickle("../input/fold_test_2.pkl")
df["fold"] = df["sequence"].apply(lambda x: folds[x])
highFP_df = pd.read_csv('../input/df_highFPNoBB.csv')
df = pd.merge(df, highFP_df[['video_id',"video_frame","highFBNoBB"]], on=["video_id","video_frame"], how='left')
df["highFBNoBB"].fillna(False, inplace=True)

df.shape

In [ ]:
data_param = {'root_dir':INPUT_DIR,'label_dir':INPUT_DIR / "labels"}
df = df.progress_apply(lambda x: util.get_path(x, data_param, infer=True), axis=1)
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df["real_bbox"] = df["annotations"].apply(lambda annots: [list(annot.values()) for annot in annots])
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

In [ ]:
df['train'] = False
df.loc[df.query("fold == 0 and (num_bbox > 0 or highFBNoBB)").index, 'train'] = True
df['train'].value_counts()

In [ ]:
# COTS per frame
df.groupby("fold").apply(lambda df: df["num_bbox"].sum() / df.shape[0])

In [ ]:
df['fold'].value_counts(normalize=True)

In [ ]:
df.head()

# Add OOF prediction

In [ ]:
df_oof = util.load_pickle("../input/wbf_pred.pkl")
df_oof.head()

In [ ]:
df['wbf_pred'] = None
df.loc[df['fold']==1, 'wbf_pred'] = df_oof['track_pred']

In [ ]:
df_pred1 = util.load_pickle("whole_pred_0204_yolov5s6_B.pkl")
df_pred2 = util.load_pickle("whole_pred_0205_yolov5m6_B_LS02.pkl")
df['pred_0204_yolov5s6_B'] = df_pred1['pred_0204_yolov5s6_B']
df['pred_0205_yolov5m6_B_LS02'] = df_pred2['pred_0205_yolov5m6_B_LS02']

In [ ]:
best_conf_dict = {
    '0204_yolov5s6_B':0.2,
    '0205_yolov5m6_B_LS02':0.3,
}

In [ ]:
iou_thr = 0.4
skip_box_thr = 0.22
wbf_models =["0204_yolov5s6_B", '0205_yolov5m6_B_LS02']
df["wbf_noTrack_pred"] = None
frame_id = 0
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=2,
)              
num_prev_bbox = None
for i, idx in enumerate(df.query("fold!=1").index):
    row = df.loc[idx]
    bboxes_l = []
    confs_l = []    
    for model_version in wbf_models:
        BEST_CONF = best_conf_dict[model_version]
        pred_col = "pred_" + model_version 
        prd_bboxs = row[pred_col]
        prd_bboxs = [p for p in prd_bboxs if p[0] >= BEST_CONF]
        bboxes, confis = [p[1:] for p in prd_bboxs], [p[0] for p in prd_bboxs]    
        bboxes = np.array(bboxes,dtype=np.float64)
        bboxes_l.append(bboxes)
        confs_l.append(confis)
    bboxes_l = np.array(bboxes_l)
    wbf_boxes, wbf_confs, _ = run_wbf(bboxes_l, confs_l, iou_thr, skip_box_thr)
    df.at[idx, "wbf_noTrack_pred"] = [[wbf_confs[i]] + wbf_boxes[i].tolist() for i in range(len(wbf_boxes))]
    
    predictions = tracking_function(tracker, frame_id, wbf_boxes, wbf_confs, 0, num_prev_bbox)
    prd_bboxs_tracking =  [[float(p) for p in pred.split(" ")] for pred in predictions]
    num_prev_bbox = len(wbf_boxes)
    df.at[idx, "wbf_pred"] = prd_bboxs_tracking

    frame_id += 1

In [ ]:
df.loc[df.query("fold==1").index, "wbf_noTrack_pred"] = df.query("fold==1")['wbf_pred']

## check score

In [ ]:
all_gt = [np.array(x) for x in  df['real_bbox']]
all_gt_m = all_gt.copy()
all_pred = [np.array(x) if x is not None else np.array([]) for x in  df['wbf_pred']]
all_pred_notrack = [np.array(x) if x is not None else np.array([]) for x in  df['wbf_noTrack_pred']]

In [ ]:
def check_score_gt(df, q):
    selected_gt = []
    selected_pred = []
    for idx in df.query(q).index:
        selected_gt.append(all_gt[idx])
        selected_pred.append(all_pred[idx])
    f2_dict = util.calc_f2_score(selected_gt, selected_pred, verbose=False) 
    return f2_dict['f2']

def check_score_gt_notrack(df, q):
    selected_gt = []
    selected_pred = []
    for idx in df.query(q).index:
        selected_gt.append(all_gt[idx])
        selected_pred.append(all_pred_notrack[idx])
    f2_dict = util.calc_f2_score(selected_gt, selected_pred, verbose=False) 
    return f2_dict['f2']

In [ ]:
check_score_gt(df, "fold==1")

In [ ]:
check_score_gt(df, "train")

In [ ]:
check_score_gt(df, "not train")

In [ ]:
check_score_gt_notrack(df, "train")

## save video

In [ ]:
# for video_id, sequence in df.query("fold!=1").groupby(["video_id","sequence"]).size().index:
#     name = f'check-{video_id}-{sequence}'
#     make_video(df, video_id, sequence, 'wbf_pred', out_dir, 0, name=name)

In [ ]:
video_id=2
sequence=22643
s_f = 5661
e_f = 5691

name = f'checkFix2-{video_id}-{sequence}_{s_f}_{e_f}'
make_video(df, video_id, sequence, 'wbf_pred', out_dir, 0, name=name,s_f=s_f, e_f=e_f)

# modify GT

## Modify fold==1 GT

In [ ]:
# 1-15827-2148-2205
video_id = 1
sequence = 15827
s_f = 2148
e_f = 2205
q = f"video_id == {video_id} and sequence == {sequence} and video_frame >= {s_f} and video_frame <= {e_f}"
df_check = df.query(q).copy()
for idx in df_check.index:
    pred = all_pred[idx]
    if len(all_gt_m[idx]) == 0:
        if len(pred) > 0:
            all_gt_m[idx] = pred[:,1:]

In [ ]:
# 2-26651-4639-4748
video_id = 2
sequence = 26651
s_f = 4639
e_f = 4748
q = f"video_id == {video_id} and sequence == {sequence} and video_frame >= {s_f} and video_frame <= {e_f}"
df_check = df.query(q).copy()
for idx in df_check.index:
    pred = all_pred[idx]
    if len(all_gt_m[idx]) == 0:
        if len(pred) > 0:
            all_gt_m[idx] = pred[:,1:]

In [ ]:
# modified f2
selected_gt = []
selected_pred = []
for idx in df.query("fold==1").index:
    selected_gt.append(all_gt_m[idx])
    selected_pred.append(all_pred[idx])
f2_dict = util.calc_f2_score(selected_gt, selected_pred, verbose=False) 
f2_dict['f2']

In [ ]:
df.query("video_frame == 6845")

In [ ]:
df.loc[[11893]]

## Modify fold != 1 GT

In [ ]:
video_id = 1
sequence = 18048
s_f = 6709
e_f = 6780

name = f'InterestingFP_wbf-{video_id}-{sequence}-{s_f}-{e_f}'
make_video(df, video_id, sequence, 'wbf_pred', out_dir, 0, name=name,s_f=s_f, e_f=e_f)

In [ ]:
video_id = 1
sequence = 8503
s_f =3891
e_f =3904

name = f'Error_TP_wbf-{video_id}-{sequence}-{s_f}-{e_f}'
make_video(df, video_id, sequence, 'wbf_pred', out_dir, 0, name=name,s_f=s_f, e_f=e_f)

# Add GT

In [ ]:
video_dir = out_dir
check_videos = glob(video_dir + "*.mp4")
gt_candidates = []
for name in check_videos:
    name = name.split("/")[-1]
    if name in ['UnkownFP_wbf-2-26651-4154-4200.mp4', 'commonFP_wbf-0-996-11971-11994.mp4']:
        continue
    if "fp" in name.lower() and "error" not in name.lower():
        print(name)
        _, video_id, sequence, s_f, e_f_mp4 = name.split("-")
        e_f = e_f_mp4[:-4]
        gt_candidates.append([int(video_id), int(sequence), int(s_f), int(e_f)])

In [ ]:
gt_candidates

In [ ]:
gt_candidates_dfs = []
cols = ["video_id","sequence","video_frame"]
for video_id, sequence, s_f, e_f in gt_candidates:
    l = e_f - s_f + 1
    df_ = pd.DataFrame([f for f in range(s_f, e_f+1)], columns=['video_frame'])
    df_['video_id'] = video_id
    df_['sequence'] = sequence
    gt_candidates_dfs.append(df_)
gt_candidates_df = pd.concat(gt_candidates_dfs)[cols]

In [ ]:
df_new = pd.merge(gt_candidates_df, df[cols + ['annotations','image_path',"real_bbox", "wbf_noTrack_pred"]], on=cols)
df_new = df_new.drop_duplicates(subset=['video_id',"sequence","video_frame"]).sort_values(["video_id","video_frame"])

In [ ]:
df_new["added_pred"] = [list([]) for _ in range(df_new.shape[0])]
df_new["new_real_bbox"] = [list([]) for _ in range(df_new.shape[0])]

In [ ]:
for idx, row in df_new.iterrows():
    if len(row['wbf_noTrack_pred']) == 0:
        continue
    elif len(row['real_bbox']) == 0:
        df_new.at[idx, 'added_pred'] = df_new.loc[idx,'wbf_noTrack_pred']
    else:
        wbf_pred = np.array(row['wbf_noTrack_pred'])
        real_gt = np.array(row['real_bbox'])
        added_pred = []
        ious = util.calc_iou(np.array(wbf_pred)[:,1:], np.array(real_gt))
        ious_max = ious.max(axis=1)
        added_pred = wbf_pred[ious_max < 0.3].copy().tolist()
        df_new.at[idx,'added_pred'] = added_pred

In [ ]:
(df_new['added_pred'].apply(len) > 0).sum()

In [ ]:
df_new['new_real_bbox'] = df_new.apply(lambda x: x['real_bbox'] + [p[1:] for p in x['added_pred']], axis=1)

In [ ]:
df_new['new_real_bbox'].iloc[20:].iloc[0][0]

In [ ]:
df_new['wbf_pred'] = df_new['new_real_bbox'].apply(lambda pp:  [[1] + p for p in pp])

In [ ]:
make_video(df_new, None, None, 'wbf_pred', out_dir, 0, name="new_GT_check",s_f=None, e_f=None)

## combine with old data

In [ ]:
if 'mark' in df.columns:
    del df['mark']
df_new['mark'] = True
df = pd.merge(df, df_new[cols + ['mark']], on=cols, how='left')
df['mark'].fillna(False, inplace=True)

In [ ]:
df["new_real_bbox"] = df['real_bbox']
df['added_pred'] = [list([]) for i in range(df.shape[0])]
df.loc[df.query('mark').index, "new_real_bbox"] = df_new['new_real_bbox'].values
df.loc[df.query('mark').index, "added_pred"] = df_new['added_pred'].values

In [ ]:
df.query("")

In [ ]:
df['num_bbox'].sum()

In [ ]:
1120/11898

In [ ]:
df['added_pred'].apply(len)

In [ ]:
df.groupby("fold").apply(lambda df: np.sum(((df['added_pred'].apply(len)>0).astype(int)  + (df['num_bbox'] == 0)).astype(int) > 1 ))

In [ ]:
353 / (df['num_bbox'] > 0).sum()

In [ ]:
df.groupby("fold").apply(lambda df: np.sum(df['added_pred'].apply(len)))

In [ ]:
# save it
def from_bbox_to_str(bbox):
    if len(bbox) == 0:
        return '[]'
    else:
        d = []
        for p in bbox:
            d.append({"x":round(p[0],1), 
                      "y":round(p[1],1), 
                      "width":round(p[2],1), 
                      "height":round(p[3],1)})
        return str(d)
def from_pred_to_str(bbox):
    if len(bbox) == 0:
        return '[]'
    else:
        d = []
        for p in bbox:
            d.append({'conf':round(p[0],3),
                      "x":round(p[1],2), 
                      "y":round(p[2],3), 
                      "width":round(p[3],4), 
                      "height":round(p[4],5)})
        return str(d)
df['new_annotations'] = df['new_real_bbox'].apply(from_bbox_to_str)
df['new_added_pred'] = df['added_pred'].apply(from_pred_to_str)

In [ ]:
ast.literal_eval(df.iloc[9316]["new_annotations"])

In [ ]:
ast.literal_eval(df.iloc[9316]["new_added_pred"])

In [ ]:
ast.literal_eval(df_origin.query("video_id == 1 and video_frame==4164").iloc[0]['annotations'])

In [ ]:
df_origin = pd.merge(df_origin, df[['video_id','video_frame','new_annotations','new_added_pred']], on=['video_id','video_frame'], how='left')
df_origin.fillna("[]", inplace=True)
df_origin

In [ ]:
df_origin.to_csv("../../data/tensorflow-great-barrier-reef/train_with_added_GT.csv",index=False)

# Improve Tracking

In [ ]:
# tracking needs to be improved! especially when the camera moves fast

In [ ]:
??Tracker

In [ ]:
iou_thr = 0.4
skip_box_thr = 0.22
wbf_models =["0204_yolov5s6_B", '0205_yolov5m6_B_LS02']
df["wbf_noTrack_pred"] = None
frame_id = 0
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=5,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=2,
)                      
for i, idx in enumerate(df.query("sequence == 18048").index):
    row = df.loc[idx]
    bboxes_l = []
    confs_l = []    
    for model_version in wbf_models:
        BEST_CONF = best_conf_dict[model_version]
        pred_col = "pred_" + model_version 
        prd_bboxs = row[pred_col]
        prd_bboxs = [p for p in prd_bboxs if p[0] >= BEST_CONF]
        bboxes, confis = [p[1:] for p in prd_bboxs], [p[0] for p in prd_bboxs]    
        bboxes = np.array(bboxes,dtype=np.float64)
        bboxes_l.append(bboxes)
        confs_l.append(confis)
    bboxes_l = np.array(bboxes_l)
    wbf_boxes, wbf_confs, _ = run_wbf(bboxes_l, confs_l, iou_thr, skip_box_thr)
    df.at[idx, "wbf_noTrack_pred"] = [[wbf_confs[i]] + wbf_boxes[i].tolist() for i in range(len(wbf_boxes))]
    
    predictions = tracking_function(tracker, frame_id, wbf_boxes, wbf_confs, 0)
    prd_bboxs_tracking =  [[float(p) for p in pred.split(" ")] for pred in predictions]
    df.at[idx, "wbf_pred"] = prd_bboxs_tracking

    frame_id += 1

In [ ]:
seq_check = 18048
df_check = df.query("sequence == @seq_check").copy()

In [ ]:
df_check["pred_track_num"] = df_check["wbf_pred"].apply(len)
df_check["pred_notrack_num"] = df_check["wbf_noTrack_pred"].apply(len)

In [ ]:
df_check[["video_frame","num_bbox","pred_track_num","pred_notrack_num"]].iloc[20:30]
## 11851 is where the things go wrong

# Similarity

In [ ]:
import image_similarity_measures
from image_similarity_measures.quality_metrics import metric_functions

In [ ]:
metric_functions

In [ ]:
df.groupby("sequence").size().sort_values()

In [ ]:
max_p = 255
seq_check = [18048,17665,44160,29424]
df_check = df.query("sequence in @seq_check").copy()

prev_image = cv2.imread(str(df.iloc[-1]['image_path']))
for idx, row in tqdm(df_check.iterrows()):
    curr_image = cv2.imread(str(row['image_path']))
    for name, func in metric_functions.items():
        if name in ['rmse']:
            sim = func(curr_image, prev_image, max_p)
        else:
            continue
        df_check.loc[idx, name] = sim
    prev_image = curr_image

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(18,9))
axes[0].plot(df_check['rmse'].values)
axes[0].twinx().plot(np.diff(df_check['sequence']) != 0, color='red' )
axes[1].plot(np.abs(np.diff(df_check['rmse'].values)))
#axes[1].twinx().plot(np.diff(df_check['sequence']) != 0, color='red' )
plt.show()

In [ ]:
prev_image = cv2.imread(str(df.iloc[-1]['image_path']))
for idx, row in tqdm(df.iterrows()):
    curr_image = cv2.imread(str(row['image_path']))
    for name, func in metric_functions.items():
        if name in ['rmse']:
            sim = func(curr_image, prev_image, max_p)
        else:
            continue
        df.loc[idx, name] = sim
    prev_image = curr_image

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(18,9))
axes[0].plot(df['rmse'].values)
axes[0].twinx().plot(np.diff(df['sequence']) != 0, color='red' )
axes[1].plot(np.abs(np.diff(df['rmse'].values)))
#axes[1].twinx().plot(np.diff(df_check['sequence']) != 0, color='red' )
plt.show()

In [ ]:
df['rmse_diff_abs'] = df['rmse'].diff().abs()
df_seq_place = df.loc[np.diff(df['sequence'], prepend=0) != 0].copy()

In [ ]:
df_seq_place[['sequence','rmse_diff_abs']]#.min()

In [ ]:
prev_img